In [1]:
#RNN을 어떻게 입력하고 어떻게 출력을 내는지!

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

C:\Users\S9304\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
#hidden_size: cell을 만들 때 출력의 크기 결정
#sequence_length: RNN cell 의 갯수
#batch_size= batch 단위로 자른 문장의 갯수


In [3]:
#어떤 입력을 넣을 것인가~?
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

![image](https://cloud.githubusercontent.com/assets/901975/23348727/cc981856-fce7-11e6-83ea-4b187473466b.png)

input dimension는 4. 배열이 4칸.


hidden size에 따라 output의 크기가 결정

In [4]:
# One cell RNN input_dim (4) -> output_dim (2)
hidden_size = 2
#1.cell을 만든다. 출력의 크기를 정해준다.
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
#2.입력을 주고 구동시켜 출력 값을 뽑아준다. cell과 data를 인자로 준다.
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

#output과 state의 마지막 값을 출력.

sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())

#초반에 랜덤하게 잡힌 weight들에 의해 나온 output.

array([[[-0.36553764, -0.34202105]]], dtype=float32)


![image](https://cloud.githubusercontent.com/assets/901975/23383634/649efd0a-fd82-11e6-925d-8041242743b0.png)

In [5]:

with tf.variable_scope('two_sequences') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32) #sequence 5개..
    print(x_data.shape)
    pp.pprint(x_data)
    #hidden size:2 sequence_length=5
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval()) #초기화된 가중치

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.4219419 ,  0.28352183],
        [ 0.31184533, -0.01762985],
        [-0.24727556, -0.07796288],
        [-0.35040453, -0.2592458 ],
        [-0.10380617,  0.45079595]]], dtype=float32)


![image](https://cloud.githubusercontent.com/assets/901975/23383681/9943a9fc-fd82-11e6-8121-bd187994e249.png)

In [6]:
#효과적으로 학습하기 위해 여러 문자열을 batch를 통해 학습시킴.
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicRNNCell(num_units=hidden_size)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
        

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.6703314 ,  0.35459587],
        [-0.07396301,  0.10975394],
        [-0.20059673,  0.48118746],
        [-0.0507838 ,  0.40923628],
        [ 0.38218033, -0.35424712]],

       [[-0.44866988,  0.08576993],
        [ 0.10032262, -0.3475839 ],
        [-0.3670809 ,  0.56332636],
        [-0.06310067,  0.3761598 ],
        [-0.06004171,  0.44161084]],

       [[-0.2319618 ,  0.50511754],
        [-0.04868553,  0.40170482],
        [-0.28093794,  0.00109422],
        [-0.51972115,  0.04668446],
        [-0.36683375,  0.44209662]]], dtype=float32)
